# NOTE:  THIS NOTEBOOK WILL TAKE 5-10 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.


# Analyze Data Quality with SageMaker Processing Jobs and Spark

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Spark are used to process and analyze data sets in order to detect data quality issues and prepare them for model training.  

In this notebook we'll use Amazon SageMaker Processing with a library called [**Deequ**](https://github.com/awslabs/deequ), and leverage the power of Spark with a managed SageMaker Processing Job to run our data processing workloads.

Here are some great resources on Deequ: 
* Blog Post:  https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/
* Research Paper:  https://assets.amazon.science/4a/75/57047bd343fabc46ec14b34cdb3b/towards-automated-data-quality-management-for-machine-learning.pdf

![Deequ](./img/deequ.png)

![Processing Job](./img/processing.jpg)

# Amazon Customer Reviews Dataset

https://s3.amazonaws.com/amazon-reviews-pds/readme.html

### Dataset Columns:

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.

In [2]:
%store -r setup_dependencies_passed

In [3]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOKS")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [4]:
print(setup_dependencies_passed)

True


# Run the Analysis Job using a SageMaker Processing Job with Spark
Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built with our Spark script.

In [5]:
import sagemaker
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'
)

# Review the Spark preprocessing script.

In [6]:
!pygmentize preprocess_deequ_pyspark.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-deps", "pydeequ==0.1.5"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas==1.1.4"])

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType
from pyspark.sql.functions import *

from pydeequ.analyzers import *
from pydeequ.checks import *
from pydeequ.verification import *
from pydeequ.suggestions import *

# PySpark Deequ GitHub Repo:  https://github.com/awslabs/python-deequ


def main():
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))

    # Retrieve the args and replace 's3://' with 's3a://' (used by Spark)
    s3_input_data = args["s3_input_data"].replace("s3://", "s3a://")
    print(s3_input_data)
    s3_output_analyze_dat

In [7]:
from sagemaker.spark.processing import PySparkProcessor

processor = PySparkProcessor(
    base_job_name="spark-amazon-reviews-analyzer",
    role=role,
    framework_version="2.4",
    instance_count=2,
    instance_type="ml.m5.2xlarge",
    max_runtime_in_seconds=300,
)

In [8]:
s3_input_data = "s3://{}/amazon-reviews-pds/tsv/".format(bucket)
print(s3_input_data)

s3://sagemaker-us-east-1-079002598131/amazon-reviews-pds/tsv/


In [9]:
!aws s3 ls $s3_input_data

2023-03-14 21:58:44   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2023-03-14 21:58:45   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz
2023-03-14 21:58:46   12134676 amazon_reviews_us_Gift_Card_v1_00.tsv.gz


## Setup Output Data

In [10]:
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = "amazon-reviews-spark-analyzer-{}".format(timestamp_prefix)
processing_job_name = "amazon-reviews-spark-analyzer-{}".format(timestamp_prefix)

print("Processing job name:  {}".format(processing_job_name))

Processing job name:  amazon-reviews-spark-analyzer-2023-03-15-01-57-05


In [11]:
s3_output_analyze_data = "s3://{}/{}/output".format(bucket, output_prefix)

print(s3_output_analyze_data)

s3://sagemaker-us-east-1-079002598131/amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output


## Start the Spark Processing Job

_Notes on not using `ProcessingInput` and `ProcessingOutput`:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by `ProcessingInput` and `ProcessingOutput` (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes.
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use `ProcessingInput`, the data will be copied to each node (which we don't want in this case since Spark already handles this)

In [12]:
from sagemaker.processing import ProcessingOutput

processor.run(
    submit_app="preprocess_deequ_pyspark.py",
    submit_jars=["deequ-1.0.3-rc2.jar"],
    arguments=[
        "s3_input_data",
        s3_input_data,
        "s3_output_analyze_data",
        s3_output_analyze_data,
    ],
    logs=True,
    wait=False,
)

INFO:sagemaker:Creating processing-job with name spark-amazon-reviews-analyzer-2023-03-15-01-57-05-822


In [13]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, processing_job_name
        )
    )
)

In [14]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After a Few Minutes</b>'.format(
            region, processing_job_name
        )
    )
)

In [15]:
from IPython.core.display import display, HTML

s3_job_output_prefix = output_prefix

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(
            bucket, s3_job_output_prefix, region
        )
    )
)

# Monitor the Processing Job

In [16]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

{'ProcessingInputs': [{'InputName': 'jars', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-079002598131/spark-amazon-reviews-analyzer-2023-03-15-01-57-05-822/input/jars', 'LocalPath': '/opt/ml/processing/input/jars', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-079002598131/spark-amazon-reviews-analyzer-2023-03-15-01-57-05-822/input/code/preprocess_deequ_pyspark.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingJobName': 'spark-amazon-reviews-analyzer-2023-03-15-01-57-05-822', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 2, 'InstanceType': 'ml.m5.2xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 300}, 'AppSpecification':

In [17]:
running_processor.wait()

...........................03-15 02:01 smspark.cli  INFO     Parsing arguments. argv: ['/usr/local/bin/smspark-submit', '--jars', '/opt/ml/processing/input/jars', '/opt/ml/processing/input/code/preprocess_deequ_pyspark.py', 's3_input_data', 's3://sagemaker-us-east-1-079002598131/amazon-reviews-pds/tsv/', 's3_output_analyze_data', 's3://sagemaker-us-east-1-079002598131/amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output']
03-15 02:01 smspark.cli  INFO     Raw spark options before processing: {'jars': '/opt/ml/processing/input/jars', 'class_': None, 'py_files': None, 'files': None, 'verbose': False}
03-15 02:01 smspark.cli  INFO     App and app arguments: ['/opt/ml/processing/input/code/preprocess_deequ_pyspark.py', 's3_input_data', 's3://sagemaker-us-east-1-079002598131/amazon-reviews-pds/tsv/', 's3_output_analyze_data', 's3://sagemaker-us-east-1-079002598131/amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output']
03-15 02:01 smspark.cli  INFO     Rendered spark options: {'jars'

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# Inspect the Processed Output 

## These are the quality checks on our dataset.

## _The next cells will not work properly until the job completes above._

In [18]:
!aws s3 ls --recursive $s3_output_analyze_data/

2023-03-15 02:02:20          0 amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/constraint-checks/_SUCCESS
2023-03-15 02:02:20        773 amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/constraint-checks/part-00000-82742180-7a0e-413f-a4e7-ee7fbf229ec4-c000.csv
2023-03-15 02:02:36          0 amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/constraint-suggestions/_SUCCESS
2023-03-15 02:02:36       8615 amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/constraint-suggestions/part-00000-cbdfbe23-dc03-4285-be40-a6a5db3dfff8-c000.csv
2023-03-15 02:02:12          0 amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/dataset-metrics/_SUCCESS
2023-03-15 02:02:11        364 amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/dataset-metrics/part-00000-dba1f2ad-31b4-47b1-9774-07c2eeabbdb3-c000.csv
2023-03-15 02:02:22          0 amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/success-metrics/_SUCCESS
2023-03-15 02:02:22        277 amazon-reviews-s

## Copy the Output from S3 to Local
* dataset-metrics/
* constraint-checks/
* success-metrics/
* constraint-suggestions/


In [19]:
!aws s3 cp --recursive $s3_output_analyze_data ./amazon-reviews-spark-analyzer/ --exclude="*" --include="*.csv"

download: s3://sagemaker-us-east-1-079002598131/amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/constraint-suggestions/part-00000-cbdfbe23-dc03-4285-be40-a6a5db3dfff8-c000.csv to amazon-reviews-spark-analyzer/constraint-suggestions/part-00000-cbdfbe23-dc03-4285-be40-a6a5db3dfff8-c000.csv
download: s3://sagemaker-us-east-1-079002598131/amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/constraint-checks/part-00000-82742180-7a0e-413f-a4e7-ee7fbf229ec4-c000.csv to amazon-reviews-spark-analyzer/constraint-checks/part-00000-82742180-7a0e-413f-a4e7-ee7fbf229ec4-c000.csv
download: s3://sagemaker-us-east-1-079002598131/amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/dataset-metrics/part-00000-dba1f2ad-31b4-47b1-9774-07c2eeabbdb3-c000.csv to amazon-reviews-spark-analyzer/dataset-metrics/part-00000-dba1f2ad-31b4-47b1-9774-07c2eeabbdb3-c000.csv
download: s3://sagemaker-us-east-1-079002598131/amazon-reviews-spark-analyzer-2023-03-15-01-57-05/output/success-metrics/part-0000

## Analyze Constraint Checks

In [20]:
import glob
import pandas as pd

def load_dataset(path, sep, header):
    data = pd.concat(
        [pd.read_csv(f, sep=sep, header=header) for f in glob.glob("{}/*.csv".format(path))], ignore_index=True
    )

    return data

In [21]:
df_constraint_checks = load_dataset(path="./amazon-reviews-spark-analyzer/constraint-checks/", sep="\t", header=0)
df_constraint_checks[["check", "constraint", "constraint_status", "constraint_message"]]

,check,constraint,constraint_status,constraint_message
0,Review Check,SizeConstraint(Size(None)),Success,NaN
1,Review Check,"MaximumConstraint(Maximum(star_rating,None))",Success,NaN
2,Review Check,UniquenessConstraint(Uniqueness(List(review_id...,Success,NaN
3,Review Check,ComplianceConstraint(Compliance(marketplace co...,Success,NaN
4,Review Check,"MinimumConstraint(Minimum(star_rating,None))",Success,NaN
5,Review Check,"CompletenessConstraint(Completeness(review_id,...",Success,NaN
6,Review Check,CompletenessConstraint(Completeness(marketplac...,Success,NaN


## Analyze Dataset Metrics

In [22]:
df_dataset_metrics = load_dataset(path="./amazon-reviews-spark-analyzer/dataset-metrics/", sep="\t", header=0)
df_dataset_metrics

,entity,instance,name,value
0,Column,review_id,Completeness,1.000000
1,Mutlicolumn,"total_votes,star_rating",Correlation,-0.086052
2,Column,star_rating,Mean,4.102493
3,Mutlicolumn,"total_votes,helpful_votes",Correlation,0.985751
4,Column,review_id,ApproxCountDistinct,381704.000000
5,Dataset,*,Size,396601.000000
6,Column,top star_rating,Compliance,0.765893


## Analyze Success Metrics

In [23]:
df_success_metrics = load_dataset(path="./amazon-reviews-spark-analyzer/success-metrics/", sep="\t", header=0)
df_success_metrics

,entity,instance,name,value
0,Column,review_id,Completeness,1.0
1,Dataset,*,Size,396601.0
2,Column,star_rating,Minimum,1.0
3,Column,marketplace,Completeness,1.0
4,Column,review_id,Uniqueness,1.0
5,Column,star_rating,Maximum,5.0
6,Column,"marketplace contained in US,UK,DE,JP,FR",Compliance,1.0


## Analyze Constraint Suggestions

In [24]:
pd.set_option('max_colwidth', 999)

df_constraint_suggestions = load_dataset(path='./amazon-reviews-spark-analyzer/constraint-suggestions/', sep='\t', header=0)
xdf_constraint_suggestions

NameError: name 'xdf_constraint_suggestions' is not defined

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}